In [82]:
from sklearn.svm import OneClassSVM
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
import pandas as pd
from joblib import dump

from os.path import join, basename, exists
from os import makedirs, listdir, mkdir
from shutil import rmtree

import numpy as np
import pyshark
from traceback import print_exc
import csv
import concurrent.futures
from joblib import load
import nest_asyncio

DATA_DIR = join("data")
PCAP_D = join(DATA_DIR, "pcap")
CSV_D = join(DATA_DIR, "csv")
MODELS_D = join(DATA_DIR, "models")
INTERVALS_D = join(CSV_D, "intervals")
ANOMALIES_D = join(CSV_D, "anomalies")

for d in (DATA_DIR, PCAP_D, CSV_D, MODELS_D, INTERVALS_D, ANOMALIES_D):
        makedirs(d, exist_ok=True)

PCAP = {"1": join(PCAP_D, "mega104-17-12-18.pcapng"),
        "2": join(PCAP_D, "10122018-104Mega.pcapng"),}

CSV = {"1": join(CSV_D, "mega104-17-12-18.csv"),
       "2": join(CSV_D, "10122018-104Mega.csv"),}

nest_asyncio.apply()

In [6]:
def parse(num=1):
    pcap_file = PCAP[str(num)]
    print(f"Reading from {pcap_file}")
    packets = pyshark.FileCapture(pcap_file)

    parsed_data = [("asdu_len", "io_type", "type_id", "src", "dst", "intervl", "relative_time_stamp")]
    
    previous = 0
    first_time_stamp = packets[0].sniff_time
    relative_time = 0
    interval = 0
    hosts = {}
    next_index = 0
    for p in packets:
        if "iec60870_104" not in [l.layer_name for l in p.layers]:
            continue
        
        # Count time from the previous IEC 104 packet
        if previous != 0:
            interval = float((p.sniff_time - previous).total_seconds())
            relative_time = (p.sniff_time - first_time_stamp).total_seconds()
        if p.ip.src not in hosts.keys():
            hosts[p.ip.src] = next_index
            next_index += 1
        if p.ip.dst not in hosts.keys():
            hosts[p.ip.dst] = next_index
            next_index += 1
        
        src = hosts[p.ip.src]
        dst = hosts[p.ip.dst]
        
        previous = p.sniff_time
        # Extract only one 'representative' for the current package
        asdu_layer = p.get_multiple_layers("iec60870_asdu")
        if len(asdu_layer) == 0:
            continue
        asdu_layer = asdu_layer[0]

        iec_header_layer = p.get_multiple_layers("iec60870_104")
        # Aggregate values if more then one header is present in the packet
        iec_header = iec_header_layer[0]
        try:
            iec_header.apdulen = int(iec_header.apdulen)
        except AttributeError:
            # Not all APDU has valid apdulen attribute. Those packets in
            # Wireshark displayed as a byte sequence, so this packet can
            # be parsed
            print("Error in converting the value in packet")
            print_exc()
            print(p)
            continue

        if len(iec_header_layer) != 1:
            for entry in iec_header_layer[1:]:
                iec_header.apdulen += int(entry.apdulen)

        try:
            if asdu_layer:
                parsed_data.append((iec_header.apdulen, asdu_layer.ioa, asdu_layer.typeid, src, dst, interval, relative_time))
        except:
            # Ignoring error if data can't be appended for some reasons.
            print("Error in parsing the packet")
            print_exc()
            print(p)

    with open(CSV[str(num)], "w") as f:
        writer = csv.writer(f)
        writer.writerows(parsed_data)

    print(f"CSV file is stored into {CSV[str(num)]}")


In [122]:
def create_model(num=1, nu=0.018, kernel = 'sigmoid'):
    
    out_dir = join(MODELS_D, kernel)
    iec104 = pd.read_csv(CSV[str(num)], header=0, skipinitialspace=True)

    iec104 = iec104.drop(columns=["relative_time_stamp"])
    x_train, x_test = train_test_split(iec104, train_size=2/3, test_size=1/3,
                                    shuffle=False, random_state=0)
    one_class_svm = OneClassSVM(nu=nu, kernel=kernel).fit(x_train)
    
    if not exists(out_dir):
        mkdir(out_dir)
    
    dump(one_class_svm, join(out_dir, f"oc-svm-{num}-nu-{nu:.3f}.joblib"))
    prediction = one_class_svm.predict(x_test)
    size = len(prediction)
    t = [i for i in prediction if i == -1]
    anomalies = len(t)
    t = [i for i in prediction if i == 1]
    ok = len(t)
    perc_anom = anomalies/size
    
    # print("-"*(len(f"Datset: {CSV[str(num)]}") + 2))
    # print(f"Datset: {CSV[str(num)]}")
    # print(f"Nu is: {nu:.4f}")
    # print(f"Total number of samples: {size}")
    # print(f"Normal: {ok} ({100*(1-perc_anom):.2f}%)")
    # print(f"Anomalies: {anomalies} ({100*perc_anom:.2f}%)")
    # print("-"*(len(f"Datset: {CSV[str(num)]}") + 2))

    return [nu, perc_anom, 1 - perc_anom]

In [138]:
def predict(model, nu, dataset, out=True, kernel="rbf"):
    model = join(MODELS_D, kernel, f"oc-svm-{model}-nu-{nu}.joblib")

    svm = load(model)
    data = pd.read_csv(dataset).drop(columns=["relative_time_stamp"])

    prediction = svm.predict(data)
    size = len(prediction)
    t = [i for i in prediction if i == -1]
    anomalies = len(t)
    t = [i for i in prediction if i == 1]
    ok = len(t)
    perc_anom = anomalies/size

    if out:
        print("-"*(len(f"Total number of samples: {size}") + 2))
        print(f"Datset: {dataset}")
        print(f"Total number of samples: {size}")
        print(f"Normal: {ok} ({100*(1-perc_anom):.2f}%)")
        print(f"Anomalies: {anomalies} ({100*perc_anom:.2f}%)")
        print("-"*(len(f"Total number of samples: {size}") + 2))
    # return prediction, (size, ok, 100*(1-perc_anom), anomalies, 100*perc_anom)
    return prediction



In [153]:
def get_interval(i, interval, model_num, type_="o"):
    f = join(INTERVALS_D, f"model-{model_num}-{interval}min", f"frame-{model_num}-{i}.csv") if type_ == "o" else join(ANOMALIES_D, f"frame-{model_num}-{i}.csv")
    assert exists(f)
    return f


In [162]:
def create_anomalies(i, interval, model_num):
    frame_name = get_interval(i, interval, model_num, type_="o")
    frame = pd.read_csv(frame_name)
    row_num = frame.shape[0]
    # generate 15 random indexes to change size of data
    min_size = frame["asdu_len"].min()
    max_size = frame["asdu_len"].max()

    indexes = np.random.randint(0, row_num, size=int(row_num*0.1))
    values = np.random.randint(min_size, max_size, int(row_num*0.1))

    frame.loc[indexes, ["asdu_len"]] = values
    
    # take range of 25 items to imitate DOS  
    index = np.random.randint(0, row_num-int(row_num*0.2))
    range_ = pd.RangeIndex(index, index + int(row_num*0.2))
    
    src = frame.loc[range_, "src"]
    frame.loc[range_, "src"] = frame.loc[range_, "dst"]
    frame.loc[range_, "dst"] = src
    frame.to_csv(join(ANOMALIES_D, basename(frame_name)), index=False)
    indexes = np.append(indexes, range_)
    # indexes.append()

    return frame, (indexes, values), index


In [89]:
def split_to_intervals(model_num, interval="5min"):
    interval_dir = join(INTERVALS_D, f"model-{model_num}-{interval}")
    if exists(interval_dir):
        rmtree(interval_dir)
    
    mkdir(interval_dir)

    data = pd.read_csv(CSV[model_num])
    data["relative_time_stamp"] = pd.to_datetime(data["relative_time_stamp"], unit='s',)
    for i, frame in enumerate(data.groupby(pd.Grouper(key="relative_time_stamp",freq=interval))):
        frame[1].to_csv(join(interval_dir, f"frame-{model_num}-{i}.csv"), index=False, date_format="%M:%S.%f")
    return interval_dir

In [47]:
# with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
#         executor.map(parse, [1, 2])
parse(1)
parse(2)

Reading from data/pcap/mega104-17-12-18.pcapng


Exception ignored in: <function Capture.__del__ at 0x7fdc2ca0f790>
Traceback (most recent call last):
  File "/home/xyadlo00/studies/FIT/MITAI/1-rocnik/letni/PDS/proj/pds-env/lib/python3.8/site-packages/pyshark/capture/capture.py", line 445, in __del__
    self.close()
  File "/home/xyadlo00/studies/FIT/MITAI/1-rocnik/letni/PDS/proj/pds-env/lib/python3.8/site-packages/pyshark/capture/capture.py", line 436, in close
    self.eventloop.run_until_complete(self.close_async())
  File "/home/xyadlo00/studies/FIT/MITAI/1-rocnik/letni/PDS/proj/pds-env/lib/python3.8/site-packages/nest_asyncio.py", line 81, in run_until_complete
    return f.result()
  File "/usr/lib/python3.8/asyncio/futures.py", line 178, in result
    raise self._exception
  File "/usr/lib/python3.8/asyncio/tasks.py", line 280, in __step
    result = coro.send(None)
  File "/home/xyadlo00/studies/FIT/MITAI/1-rocnik/letni/PDS/proj/pds-env/lib/python3.8/site-packages/pyshark/capture/capture.py", line 440, in close_async
    awa

KeyboardInterrupt: 

In [11]:
# One-shot training
nu = 0.017
create_model(1, nu)
create_model(2, nu)

---------------------------------------
Datset: data/csv/mega104-17-12-18.csv
Nu is: 0.0170
Total number of samples: 12554
Normal: 12261 (97.67%)
Anomalies: 293 (2.33%)
---------------------------------------
---------------------------------------
Datset: data/csv/10122018-104Mega.csv
Nu is: 0.0170
Total number of samples: 22126
Normal: 21712 (98.13%)
Anomalies: 414 (1.87%)
---------------------------------------


[0.017, 0.018711018711018712, 0.9812889812889813]

In [124]:
# Check different Nu parameter for the training of the data
nu = 0.002
result_pd_1 = []
result_pd_2 = []
nus = np.arange(0.005, 0.02, 0.001)

def train(nu):
    for kernel in ["rbf", "sigmoid"]:
        entry = (nu, kernel, *create_model(1, nu=nu, kernel=kernel)[1:])
        result_pd_1.append(entry)
        entry = (nu, kernel, *create_model(2, nu=nu, kernel=kernel)[1:])
        result_pd_2.append(entry)

with concurrent.futures.ThreadPoolExecutor(max_workers=len(nus)) as executor:
        executor.map(train, nus)


df_1 = pd.DataFrame(result_pd_1, columns=["nu", "kernel", "anomalies_1", "ok_1"])
df_2 = pd.DataFrame(result_pd_2, columns=["nu", "kernel", "anomalies_2", "ok_2"])
df_1.to_csv(join(CSV_D, "pandas-df-1.csv"))
df_2.to_csv(join(CSV_D, "pandas-df-2.csv"))
df_1_sort = df_1.sort_values(by=['ok_1'], ascending=False).reset_index(drop=True)
df_2_sort = df_2.sort_values(by=["ok_2"], ascending=False).reset_index(drop=True)

ok_1 = df_1_sort.loc[(df_1_sort["ok_1"] > 0.98)]
ok_2 = df_2_sort.loc[(df_2_sort["ok_2"] > 0.98)]

merged = pd.merge(left=ok_1, right=ok_2, how="right", on="nu")
print(merged)

       nu kernel_x  anomalies_1      ok_1 kernel_y  anomalies_2      ok_2
0   0.005  sigmoid     0.004301  0.995699  sigmoid     0.006011  0.993989
1   0.005      rbf     0.004540  0.995460  sigmoid     0.006011  0.993989
2   0.006  sigmoid     0.005178  0.994822  sigmoid     0.007051  0.992949
3   0.006      rbf     0.005337  0.994663  sigmoid     0.007051  0.992949
4   0.007      rbf     0.005895  0.994105  sigmoid     0.008180  0.991820
5   0.007  sigmoid     0.006054  0.993946  sigmoid     0.008180  0.991820
6   0.008      rbf     0.006452  0.993548  sigmoid     0.009220  0.990780
7   0.008  sigmoid     0.007010  0.992990  sigmoid     0.009220  0.990780
8   0.009      rbf     0.007567  0.992433  sigmoid     0.010079  0.989921
9   0.009  sigmoid     0.008603  0.991397  sigmoid     0.010079  0.989921
10  0.010      rbf     0.008921  0.991079  sigmoid     0.011570  0.988430
11  0.010  sigmoid     0.009877  0.990123  sigmoid     0.011570  0.988430
12  0.011      rbf     0.009559  0.990

In [19]:
_, (ind, vals), i = create_anomalies(8, model_num)

In [9]:
frame_num = 3
_, (ind, vals), i = create_anomalies(frame_num)
frame_anom = get_interval(frame_num, type_='a')

res = predict(num, 0.017, frame_anom)
anoms_an = [i for i, n in enumerate(res) if n == -1]
all_data = pd.read_csv(frame_anom)

anoms_real = sorted([i for i in anoms_an if i not in anoms_ok])
print(f"Created {len(ind)}: {sorted(ind)}")
print(f"Detected {len(anoms_real)}: {anoms_real}")
print(f"{len(ind)/len(anoms_real):.02f}%")

------------------------------
Datset: data/csv/anomalies/frame-2-3.csv
Total number of samples: 981
Normal: 915 (93.27%)
Anomalies: 66 (6.73%)
------------------------------
Created 30: [17, 24, 130, 171, 195, 216, 248, 256, 428, 442, 525, 556, 559, 596, 639, 643, 650, 666, 689, 794, 795, 796, 797, 798, 799, 800, 801, 802, 803, 835]
Detected 29: [17, 24, 130, 171, 195, 216, 248, 256, 428, 442, 525, 556, 559, 596, 639, 643, 650, 666, 689, 794, 795, 796, 797, 798, 799, 800, 801, 802, 835]
1.03%


### Accuracy of original datasets

In [64]:
data_1 = pd.read_csv(CSV["1"])
data_1["relative_time_stamp"] = pd.to_datetime(data_1["relative_time_stamp"], unit='s',)
real_labels = np.ones(data_1.shape[0], dtype=int)
gen_labels = predict("1", 0.021, CSV["1"])
acc = accuracy_score(real_labels, gen_labels)
print(acc)

--------------------------------
Datset: data/csv/mega104-17-12-18.csv
Total number of samples: 37660
Normal: 36679 (97.40%)
Anomalies: 981 (2.60%)
--------------------------------
0.973951141795008


In [65]:
data_2 = pd.read_csv(CSV["2"])
data_2["relative_time_stamp"] = pd.to_datetime(data_2["relative_time_stamp"], unit='s',)
real_labels = np.ones(data_2.shape[0], dtype=int)
gen_labels = predict("2", nu, CSV["2"])
acc = accuracy_score(real_labels, gen_labels)
print(acc)

--------------------------------
Datset: data/csv/10122018-104Mega.csv
Total number of samples: 66377
Normal: 64714 (97.49%)
Anomalies: 1663 (2.51%)
--------------------------------
0.9749461409825693


In [128]:
summary = []
nu = 0.005
for kernel in ["rbf", "sigmoid"]:
    for interval in [5, 7, 10]:
        for model_num in ['1', '2']:
            
            intervals_dir = split_to_intervals(model_num, f"{interval}min")

            # Prediction
            frames = [f for f in listdir(intervals_dir) if f"frame-{model_num}" in f]
            result_acc = []
            for f in frames:
                df = pd.read_csv(join(intervals_dir, f))
                real_labels = np.ones(df.shape[0], dtype=int)
                gen_labels = predict(model_num, nu, join(intervals_dir, f), out=False, kernel=kernel)
                result_acc.append((join(intervals_dir, f), accuracy_score(real_labels, gen_labels)))
            
            result_acc = pd.DataFrame(result_acc, columns=["dataframe", "accuracy"])
            max_acc = result_acc[result_acc.accuracy == result_acc.accuracy.max()]
            tmp = (model_num, kernel, interval,
                    result_acc.accuracy.min(),
                    max_acc.shape[0],
                    (max_acc.shape[0]/result_acc.shape[0])*100,
                    result_acc.accuracy.mean())
            summary.append(tmp)
summary = pd.DataFrame(summary, columns=["model number", "kernel", "interval", "min acc",  "num 100 acc", "% 100 acc", "mean acc"])

print(summary.groupby("interval"))


In [131]:
grouped_df = summary.groupby("interval")
for key, item in grouped_df:
    # grouped_df.get_group(key).to_latex(index=False)
    print(grouped_df.get_group(key), "\n\n")

  model number   kernel  interval   min acc  num 100 acc  % 100 acc  mean acc
0            1      rbf         5  0.923077          655  80.269608  0.994372
1            2      rbf         5  0.840619            1   1.694915  0.848549
6            1  sigmoid         5  0.897436          651  79.779412  0.995043
7            2  sigmoid         5  0.988117            1   1.694915  0.994532 


  model number   kernel  interval   min acc  num 100 acc  % 100 acc  mean acc
2            1      rbf         7  0.941176          422  72.384220  0.994396
3            2      rbf         7  0.840260            1   2.380952  0.848745
8            1  sigmoid         7  0.921569          420  72.041166  0.995030
9            2  sigmoid         7  0.991520            1   2.380952  0.994550 


   model number   kernel  interval   min acc  num 100 acc  % 100 acc  mean acc
4             1      rbf        10  0.938776          263  64.460784  0.994397
5             2      rbf        10  0.842466            

In [165]:
model_num = "1"
interval = 10
nu = 0.005
kernel = "sigmoid"
anom_count = 45 # 20 for random changes + 25 serial 
interval_dir = join(INTERVALS_D, f"model-{model_num}-{interval}min")
all_frames = [f for f in listdir(interval_dir) if f"frame-{model_num}" in f]
indexes = [0, 4, 10, 13, 30, 100, 111, 222, 403]
print(f"Random indexes of frames: {indexes}")

summary = []
for i in indexes:
    print(get_interval(i, interval, model_num))
    anom_frame, _, _ = create_anomalies(i, interval, model_num)
    ok_frame = pd.read_csv(get_interval(i, interval, model_num))
    real = np.ones(ok_frame.shape[0])

    first_prediction = predict(model_num, nu=nu, dataset=ok_frame, kernel=kernel, out=False)
    prediction = predict(model_num, nu=nu, dataset=anom_frame, kernel=kernel, out=False)
    summary.append((accuracy_score(real, first_prediction),
                    1 - accuracy_score(real, first_prediction),
                    accuracy_score(real, prediction),
                    1 - accuracy_score(real, prediction)))
summary = pd.DataFrame(summary, columns=[])

Random indexes of frames: [0, 4, 10, 13, 30, 100, 111, 222, 403]
data/csv/intervals/model-1-10min/frame-1-0.csv


KeyError: '[127] not in index'